#Instalações e Imports
RODAR TODA HORA ANTES DO CODIGO
20 linhas


In [ ]:
#to use selenium in colab:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install requests

In [ ]:
import requests
import json
from selenium import webdriver
import time
import pandas as pd
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
import datetime
import requests
import json
import sys
from datetime import datetime, timezone
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from gspread.utils import rowcol_to_a1
from operator import itemgetter
from enum import IntEnum
from bs4 import BeautifulSoup

In [ ]:
 class forca(IntEnum):
    COMPRA_FORTE = -2
    COMPRA = -1
    VIÉS_NEUTRO = 0
    VENDA = 1
    VENDA_FORTE = 2

#Preparando a planilha:
Rodar toda vez antes do codigo 21 Linhas




In [ ]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
try:
  worksheet = gc.open('Crypto Analizer')
except:
  sh = gc.create('Crypto Analizer')
  worksheet = gc.open('Crypto Analizer')
sheets = worksheet.worksheets()
precisa = ['Hora','Dia','Semana']
for element in sheets:
  try:
    precisa.remove(element.title)
  except:
    pass
for item in precisa:
  worksheet.add_worksheet(item,1000,26)
#Criação das paginas inexistentes
       

In [ ]:
precisa = ['Hora','Dia','Semana']
stora = {}
for element in precisa:
  stora[element] =  worksheet.worksheet(element)

#Requests funcionando lindamente
Pegar as informações das criptos com maior crescimento no prazo de um dia 11Linhas




In [ ]:
headers = {'content-type': 'application/json'}
data = {"filter":[{"left":"name","operation":"nempty"},{"left":"exchange","operation":"equal","right":"BINANCE"},{"left":"Recommend.All|1W","operation":"in_range","right":[0.5,1]},{"left":"Recommend.All|1W","operation":"nequal","right":0.5},{"left":"name,description","operation":"match","right":"BUSD"}],"options":{"lang":"pt"},"symbols":{"query":{"types":[]},"tickers":[]},"columns":["base_currency_logoid","currency_logoid","name","close|1W","change|1W","change_abs|1W","high|1W","low|1W","volume|1W","Recommend.All|1W","exchange","description","name","type","subtype","update_mode|1W","pricescale","minmov","fractional","minmove2"],"sort":{"sortBy":"name","sortOrder":"asc"},"range":[0,150]}

r = requests.post("https://scanner.tradingview.com/crypto/scan", data=json.dumps(data), headers=headers)
retorno = json.loads(r.text)
linhas = retorno['totalCount'] + 1
retorno = retorno['data']
#Limpando os dados
dados = []
for element in retorno:
  del element['s']
  dados.append(element['d'][2])

#Extração por selenium
Para parar clicke 2x no botão de cancelar ou resete o runtime
144 linhas


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)
while True:
  #try:
  if True:
    coins = ['ADABUSD','BCHBUSD','BTCBUSD','DOGEBUSD','DOTBUSD','EOSBUSD','ETCBUSD','ETHBUSD','LINKBUSD','LTCBUSD','XRPBUSD','YFIBUSD']
    coins.extend(dados)
    coins = sorted(set(coins))
    variacoes = {}
    precodi = []
    precose = []
    precohr = []
    for element in coins:
      coindi = {}
      coinse = {}
      coinhr = {} 
      verificadodi = False
      verificadose = False
      verificadomes = False
      verificadohr = False
      wd.get(f"https://br.tradingview.com/symbols/{element}/technicals/")
      time.sleep(1)
      #Diario
      a = wd.find_elements_by_class_name('speedometerSignal-DPgs-R4s')
      nome = list(element)
      nome = nome[:-4]
      nome = ''.join(nome)
      coindi['Nome'] = nome
      coindi['Osciladores'] = a[0].text.replace(' ','_')
      coindi['Médias Moveis'] = a[2].text.replace(' ','_')
      coindi['Geral'] = a[1].text.replace(' ','_')
      coindi['Preço Atual'] = (wd.find_element_by_class_name('tv-symbol-price-quote__value').text).replace('.',',')
      coindi['Variação Diaria'] = (wd.find_element_by_class_name('js-symbol-change-pt').text).replace('(','').replace(')','').replace('.',',').replace('%','')
      coindi['Indicadores de compra'] = len(wd.find_elements_by_class_name('buyColor-2-juHm8n'))
      coindi['Indicadores de estabilidade'] = len(wd.find_elements_by_class_name('neutralColor-2-juHm8n'))
      coindi['Indicadores de venda'] = len(wd.find_elements_by_class_name('sellColor-2-juHm8n'))
      #print('{} indicadores diarios {}'.format(nome,(coindi['Indicadores de compra'] + coindi['Indicadores de estabilidade'] + coindi['Indicadores de venda'])))
      if coindi['Indicadores de compra'] + coindi['Indicadores de estabilidade'] + coindi['Indicadores de venda'] >= 17:
        verificadodi = True
        #print('verificou')
      else:
        #print('{} Descartada'.format(nome))
        continue
        
      #Semanal
      a = wd.find_elements_by_class_name('button-1cy7XKgV')
      for element in a:
        if element.text == '1 semana':
          element.click()
          break
      time.sleep(0.8)
      b = wd.find_elements_by_class_name('speedometerSignal-DPgs-R4s')
      coinse['Nome'] = nome
      coinse['Osciladores'] = b[0].text.replace(' ','_')
      coinse['Médias Moveis'] = b[2].text.replace(' ','_')
      coinse['Geral'] = b[1].text.replace(' ','_')
      coinse['Preço Atual'] = wd.find_element_by_class_name('tv-symbol-price-quote__value').text
      coinse['Variação Diaria'] = (wd.find_element_by_class_name('js-symbol-change-pt').text).replace('(','').replace(')','').replace('.',',').replace('%','')
      coinse['Indicadores de compra'] = len(wd.find_elements_by_class_name('buyColor-2-juHm8n'))
      coinse['Indicadores de estabilidade'] = len(wd.find_elements_by_class_name('neutralColor-2-juHm8n'))
      coinse['Indicadores de venda'] = len(wd.find_elements_by_class_name('sellColor-2-juHm8n'))
      #print('{} indicadores {}'.format(nome,(coinse['Indicadores de compra'] + coinse['Indicadores de estabilidade'] + coinse['Indicadores de venda'])))
      if coinse['Indicadores de compra'] + coinse['Indicadores de estabilidade'] + coinse['Indicadores de venda'] >= 17:
        verificadose = True
        #print('verificou')
      else:
        #print('Descartada'.format(nome))
        continue
        
      #hora
      c = wd.find_elements_by_class_name('button-1cy7XKgV')
      for element in c:
        if element.text == '1 hora':
          element.click()
          break
      time.sleep(0.8)
      d = wd.find_elements_by_class_name('speedometerSignal-DPgs-R4s')
      coinhr['Nome'] = nome
      coinhr['Osciladores'] = d[0].text.replace(' ','_')
      coinhr['Médias Moveis'] = d[2].text.replace(' ','_')
      coinhr['Geral'] = d[1].text.replace(' ','_')
      coinhr['Preço Atual'] = wd.find_element_by_class_name('tv-symbol-price-quote__value').text
      coinhr['Variação Diaria'] = (wd.find_element_by_class_name('js-symbol-change-pt').text).replace('(','').replace(')','').replace('.',',').replace('%','')
      coinhr['Indicadores de compra'] = len(wd.find_elements_by_class_name('buyColor-2-juHm8n'))
      coinhr['Indicadores de estabilidade'] = len(wd.find_elements_by_class_name('neutralColor-2-juHm8n'))
      coinhr['Indicadores de venda'] = len(wd.find_elements_by_class_name('sellColor-2-juHm8n'))
      #print('{} indicadores {}'.format(nome,(coinhr['Indicadores de compra'] + coinhr['Indicadores de estabilidade'] + coinhr['Indicadores de venda'])))
      if coinhr['Indicadores de compra'] + coinhr['Indicadores de estabilidade'] + coinhr['Indicadores de venda'] >= 17:
        verificadohr = True
        #print('verificou')
      else:
        #print('Descartada'.format(nome))
        continue
      if verificadose and verificadodi and verificadohr:
        precohr.append(coinhr)
        precodi.append(coindi)
        precose.append(coinse)
        #print('foi para tabela')
    #Armazenando variações:
    planilhas = [precohr,precodi,precose]
    for item in planilhas:
      for element in item:
        element['Osciladores'] = forca[element['Osciladores']]
        element['Médias Moveis'] = forca[element['Médias Moveis']]
        element['Geral'] = forca[element['Geral']]
      item.sort(key=itemgetter('Geral','Osciladores','Médias Moveis','Indicadores de venda','Indicadores de estabilidade'))
      for element in item:
        element['Osciladores'] = element['Osciladores'].name
        element['Médias Moveis'] = element['Médias Moveis'].name
        element['Geral'] = element['Geral'].name

    variacoes = {'Hora':precohr,
                 'Dia':precodi,
                 'Semana':precose}
    for table,value in zip(variacoes.keys(),variacoes.values()):
      rows = len(value)
      cols = len(value[0].keys())
      cell_stop = rowcol_to_a1(rows, cols)
      cell_range = f'A1:{cell_stop}'
      worksheet = stora[table]
      lista = []
      lista.extend(value[0].keys())
      for element in value:
        lista.extend(element.values())
      auth.authenticate_user()
      gc = gspread.authorize(GoogleCredentials.get_application_default())
      #Limpar a planilha anterior
      rowscl = len(value) + 20
      colscl = len(value[0].keys())
      cell_stopcl = rowcol_to_a1(rowscl, colscl)
      cell_rangecl = f'A1:{cell_stopcl}'
      cell_listcl = worksheet.range(cell_rangecl)
      for cellcl in cell_listcl:
        cellcl.value = ''
      worksheet.update_cells(cell_listcl)
      #Voltar a planilha
      cell_list = worksheet.range(cell_range)
      for cell,item in zip(cell_list,lista):
        cell.value = item
      worksheet.update_cells(cell_list)
      timehj = datetime.now()
      tempo = f'{timehj.hour-3}:{timehj.minute}:{timehj.second}'
      print(f'Planilha de {table} atualizada as {tempo}')

    time.sleep(10)

# testes



In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)


In [ ]:
wd.get('https://fundamentus.com.br/detalhes.php?papel=VALE3')
a = wd.page_source
bbs4b = BeautifulSoup(a,'lxml')
tablesb = bbs4b.find_all('table')
wd.fullscreen_window()
wd.save_screenshot('/home/file.png')



True

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
r = requests.get(f'https://fundamentus.com.br/detalhes.php?papel=VALE3', headers=headers)
bbs4 = BeautifulSoup(r.text,'lxml')
tablesa = bbs4.find_all('table')

In [ ]:
print(a == r.text)

False


In [ ]:
print(r.text)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html lang="pt-br">
<head>
	<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">
<title>FUNDAMENTUS - VALE3 - Invista consciente - Indicadores Fundamentalistas</title>
  <script async src="//cmp.optad360.io/items/4515d666-9434-4529-a3b5-b5379de7687f.min.js"></script>
  <script async src="//get.optad360.io/sf/ae5f21bc-5dd2-44bc-bc23-b89c8ad130a4/plugin.min.js"></script>	

	<link rel="stylesheet" href="css/estilo.css" type="text/css" media="screen, projection">
	<link rel="stylesheet" href="css/print.css" type="text/css" media="print">
	<link rel="shortcut icon" href="img/fundamentus.ico" type="image/x-icon">
	<!--[if lte IE 6]>
		<link rel="stylesheet" type="text/css" href="css/menu_ie6.css">
		<script type="text/javascript" src="script/ADxMenu.js"></script>
	<![endif]-->
	
  <!-- <script src="script/mootools.svn.js" type="text/javascript"></script>  -->
<script src="//ajax.goo